In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay

import MetaTrader5 as mt5
from tqdm import tqdm

import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller
from pandas.tseries.offsets import BDay

import csv
import time
import math
from random import randint
from itertools import combinations
from datetime import datetime, timedelta

import requests
import json

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes',  facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [ ]:
# Twitter notification keys

## Change
chat_id = "6208180231"
api_key = "6274819941:AAF8U6FnGv5A0DcSBNXCpnPCJPciwSww8-A"

In [ ]:
def send_telegram_message(message: str, chat_id: str, api_key: str,):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
                'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'
    
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response

In [ ]:
def get_bars(symbol, start_date, end_date, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bars = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    df_bars = pd.DataFrame(bars)
    df_bars["time"] = pd.to_datetime(df_bars["time"], unit="s")
    return df_bars

In [ ]:
def get_latest_bar(symbol, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bar = mt5.copy_rates_from(
        symbol, 
        timeframe, 
        datetime.now(), 
        1)  # number of bars
    df_bar = pd.DataFrame(bar)
    df_bar["time"] = pd.to_datetime(df_bar["time"], unit="s")
    return df_bar

In [ ]:
def get_prices(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol)
            
    return symbol_list

In [ ]:
def new_bar_event(symbol, last_bar):
    mt5.initialize()
    
    # Fixing a formatting error when getting last_bar['time'].iloc[0]
    date_str = last_bar['time'].iloc[0]
    date_format = '%Y-%m-%d %H:%M:%S'

    last_bar_time_formatted = datetime.strptime(date_str, date_format)
    while True:
        new_bar = get_latest_bar(symbol)        
        if new_bar['time'].iloc[0] != last_bar_time_formatted:
            return new_bar
        
        time.sleep(10)

In [ ]:
def get_num_of_lots(name, leverage=30, risk=0.01):
    ask_per_fraction = (mt5.symbols_get(name)[0].ask * mt5.symbols_get(name)[0].trade_contract_size) * mt5.symbols_get(name)[0].volume_min
    ask_per_fraction_m = ask_per_fraction / leverage
    capital_at_risk = mt5.account_info().balance * risk
    
    order_size = math.floor(capital_at_risk / ask_per_fraction_m) * mt5.symbols_get(symbol)[0].volume_min
    return  order_size

In [ ]:
def find_filling_mode(symbol):
    mt5.initialize()
    
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
 
        result = mt5.order_check(request)
            
        if result.comment == "Done":
            break
 
    return i

In [ ]:
def open_long(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [ ]:
def close_long(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [ ]:
def open_short(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [ ]:
def close_short(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
        
    result = mt5.order_send(request)   # Sends order to broker
    return result  

In [ ]:
def get_pairs(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol)
            
    return list(combinations(symbol_list, 2))

In [ ]:
def get_pairs_close(symbol_pair, start_date, end_date):
    mt5.initialize()
    
    close0 = get_bars(symbol_pair[0].name, start_date, end_date)['close']
    close1 = get_bars(symbol_pair[1].name, start_date, end_date)['close']
    
    if close0.empty or close1.empty:
        return pd.Series([], dtype='float64')
    else:
        pairs_close_data = pd.concat([close0, close1], axis=1, keys = ['close0', 'close1']).dropna()
        return pairs_close_data

In [ ]:
def first_cointegration_test(pair_close_data):
    COINTEGRATION_CONFIDENCE_LEVEL = 99
    result = coint_johansen(pair_close_data, 0, 1)
    
    confidence_level_cols = {
        90: 0,
        95: 1,
        99: 2
    }
    confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]
    
    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
    
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        return True
    else:
        return False

In [ ]:
# Engle-Granger test
def second_cointegration_test(pair_close_data):
    # Step 1: Perform OLS regression of one time series on the other
    y = pair_close_data.iloc[:, 0]
    x = sm.add_constant(pair_close_data.iloc[:, 1])
    model = sm.OLS(y, x)
    results = model.fit()
    
    # Step 2: Test residuals for stationarity using ADF test
    residuals = y - results.predict(x)
    adf_result = adfuller(residuals)
    
    # Step 3: If residuals are stationary, the two time series are considered cointegrated
    if adf_result[1] < 0.01:
        return results.params[1]
    else:
        return None


In [ ]:
def compute_Hc(time_series, max_lag=20):
        """Returns the Hurst Exponent of the time series"""
    
        time_series = time_series.to_numpy()
        lags = range(2, max_lag)

        # variances of the lagged differences
        tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]
        
        # calculate the slope of the log plot -> the Hurst Exponent
        reg = np.polyfit(np.log(lags), np.log(tau), 1)

        return reg[0]

In [ ]:
def first_sieve(all_pairs, start_date, end_date):
    candidate_stationary_pairs = []
    for pair in tqdm(all_pairs):
        pairs_close_data = get_pairs_close(pair, start_date, end_date)
        if pairs_close_data.empty:
            continue
        else:
            if first_cointegration_test(pairs_close_data):
                candidate_stationary_pairs.append(pair)
                
    return candidate_stationary_pairs

In [ ]:
def second_sieve(candidate_stationary_pairs, start_date, end_date):
    stationary_pairs = []
    for pair in tqdm(candidate_stationary_pairs):
        pairs_close_data = get_pairs_close(pair, start_date, end_date)
        
        if pairs_close_data.empty:
            continue
        else:
            slope = second_cointegration_test(pairs_close_data) 
            if slope:
                stationary_pairs.append({'coint_pair': pair, 'slope' : slope})
                
    return stationary_pairs

In [ ]:
def third_seive(stationary_pairs, start_date, end_date):
    consistent_stationary_pairs = []
    
    for pair in tqdm(stationary_pairs):
        stationary_pair = get_pairs_close(pair["coint_pair"], start_date, end_date)
        artificial_asset = stationary_pair['close0'] - pair['slope'] * stationary_pair['close1']
        
        if compute_Hc(artificial_asset) < 0.5:
            consistent_stationary_pairs.append(pair)
                
    return consistent_stationary_pairs  

In [ ]:
def rolling_forward_test(previous_prices, todays_prices, penalty1, penalty2):
    
    #print(previous_prices)
    #print(todays_prices)
    
    account  = 1000
    risk = 0.005
    
    
    rolling_interval = 100
    test = previous_prices
    test[test.size] = todays_prices[0]
    
    # Positions are encoded as follows
    HIGH = 1
    VHIGH = 2
    FREE = 0
    LOW = -1
    VLOW = -2
    
    
    position_type = 0
    wins = 0
    losses = 0

    for i in tqdm(range(1, len(todays_prices))):
        
        # Calculate trade thresholds
        #rolling_mean = apply_kalman_filter(test.rolling(rolling_interval).mean().dropna())
        #rolling_std = apply_kalman_filter(test.rolling(rolling_interval).std().dropna())
        
        rolling_mean = test.rolling(rolling_interval).mean().dropna()
        rolling_std = test.rolling(rolling_interval).std().dropna()
        
                                   
        tp = rolling_mean
        open_high = rolling_mean + rolling_std
        open_low = rolling_mean - rolling_std 
        
        
        tp_high = rolling_mean + rolling_std
        tp_low = rolling_mean - rolling_std
        
        open_vhigh = rolling_mean + 2 * rolling_std
        open_vlow = rolling_mean - 2 * rolling_std 
        
        #sl_high = rolling_mean + 2 * rolling_std
        #sl_low = rolling_mean - 2 * rolling_std
        
        sl_vhigh = rolling_mean + 3 * rolling_std
        sl_vlow = rolling_mean - 3 * rolling_std
    
        # Trade!
        if position_type == FREE:
            if todays_prices[-2] < open_high.iloc[-2] and todays_prices[-1] >= open_high.iloc[-1]:
                # Long first Short last
                position_type = HIGH
                account -= penalty2
            elif todays_prices[-2] > open_low.iloc[-2] and todays_prices[-1] <= open_low.iloc[-1]:
                # Short first Long last
                position_type = LOW
                account -= penalty1
                
        elif position_type == HIGH:
            if todays_prices[-2] > tp.iloc[-2] and todays_prices[-1] <= tp.iloc[-1]:
                # Long first Short last
                wins += 1
                account += account * risk 
                position_type = FREE
            elif todays_prices[-2] < open_vhigh.iloc[-2] and todays_prices[-1] >= open_vhigh.iloc[-1]:
                position_type = VHIGH
                account -= penalty2

        elif position_type == LOW:
            if todays_prices[-2] < tp.iloc[-2] and todays_prices[-1] >= tp.iloc[-1]:
                # Long first Short last
                wins += 1
                account += account * risk
                position_type = FREE
            elif todays_prices[-2] > open_vlow.iloc[-2] and todays_prices[-1] <= open_vlow.iloc[-1]:
                position_type = VLOW
                account -= penalty1
         
        elif position_type == VHIGH:
            if todays_prices[-2] > tp_high.iloc[-2] and todays_prices[-1] <= tp_high.iloc[-1]:
                # Long first Short last
                wins += 1
                account += account * risk
                position_type = FREE
            elif todays_prices[-2] < sl_vhigh.iloc[-2] and todays_prices[-1] >= sl_vhigh.iloc[-1]:
                account -= account * risk 
                losses += 1
                position_type = FREE
            
        elif position_type == VLOW:
            if todays_prices[-2] < tp_low.iloc[-2] and todays_prices[-1] >= tp_low.iloc[-1]:
                # Long first Short last
                wins += 1
                account += account * risk 
                position_type = FREE
            elif todays_prices[-2] > sl_vlow.iloc[-2] and todays_prices[-1] <= sl_vlow.iloc[-1]:
                account -= account * risk 
                losses += 1
                position_type = FREE
                
        if account <= 950:
            print("BROKEN")
            break
        
        test[test.size] = todays_prices[-1]
        
        
    plt.figure().set_figwidth(15)
    plt.plot(test)
    plt.plot(tp, color='k')
    plt.plot(open_high, color='y')
    plt.plot(open_low, color='y')
    plt.plot(open_vhigh, color='g')
    plt.plot(open_vlow, color='g')
    plt.plot(sl_vhigh, color='r')
    plt.plot(sl_vlow, color='r')
    plt.show()

    print(f'WINS: {wins}, LOSSES: {losses}')
    print(f'ACCOUNT BALANCE: {account}')
    print()
    return account, wins, losses
        
    

In [ ]:
location = "Retail\\Forex"
all_pairs = get_pairs(location)

# Monte Carlo simulation
      

t_end_date = datetime.now()
t_start_date = t_end_date - timedelta(days=1)  
y_end_date = t_start_date
y_start_date = y_end_date - timedelta(days=5)



print(t_end_date)
print(t_start_date)
print(y_end_date)
print(y_start_date)

#print(t_end_date)
#print(t_start_date)




In [ ]:
# Run a backtest

for index in range(len(y_consistent_pairs)):

    y_consistent_pair = get_pairs_close(y_consistent_pairs[index]["coint_pair"], y_start_date, y_end_date)
    beta = y_consistent_pairs[index]["slope"]
    y_artificial_asset = y_consistent_pair["close0"] - beta * y_consistent_pair["close1"]

    t_consistent_pair = get_pairs_close(y_consistent_pairs[index]["coint_pair"], t_start_date, t_end_date)
    t_artificial_asset = t_consistent_pair["close0"] - beta * t_consistent_pair["close1"]    

    print(f'PAIR: {y_consistent_pairs[index]["coint_pair"][0].name}, {y_consistent_pairs[index]["coint_pair"][1].name}')
    
    account = 1000
    risk = 0.05
    capital_at_risk = account * risk
    account, wins, losses = rolling_forward_test(y_artificial_asset, t_artificial_asset, 0, 0)

In [90]:
def trade_enginet(data, name_1, name_2, lot_size_1, lot_size_2, position_type):
    
    rolling_interval = 100
    
    # Positions are encoded as follows
    HIGH = 1
    VHIGH = 2
    FREE = 0
    LOW = -1
    VLOW = -2
            
    rolling_mean = data.rolling(rolling_interval).mean().dropna()
    rolling_std = data.rolling(rolling_interval).std().dropna()
                         
    tp = rolling_mean
    open_high = rolling_mean + rolling_std
    open_low = rolling_mean - rolling_std 
        
    tp_high = rolling_mean + rolling_std
    tp_low = rolling_mean - rolling_std
        
    open_vhigh = rolling_mean + 2 * rolling_std
    open_vlow = rolling_mean - 2 * rolling_std 
        
    sl_vhigh = rolling_mean + 3 * rolling_std
    sl_vlow = rolling_mean - 3 * rolling_std
    
    # Trade!
    if position_type == FREE:
        if todays_prices[-2] < open_high.iloc[-2] and todays_prices[-1] >= open_high.iloc[-1]:
            # Short first Long last
            open_short(name_1, lot_size_1)
            open_long(name_2, lot_size_2)
            send_telegram_message(f'Openned High Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Openned High Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = HIGH
            
        elif todays_prices[-2] > open_low.iloc[-2] and todays_prices[-1] <= open_low.iloc[-1]:
            # Long first Short last
            open_short(name_2, lot_size_2)
            open_long(name_1, lot_size_1)
            send_telegram_message(f'Openned Low Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Openned Low Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = LOW
                
    elif position_type == HIGH:
        if todays_prices[-2] > tp.iloc[-2] and todays_prices[-1] <= tp.iloc[-1]:
            position_1 = mt5.positions_get()[0]
            position_2 = mt5.positions_get()[1]
            
            if position_1.type == 0:
                close_long(name_2, lot_size_2, position.ticket)
            elif position_1.type == 1:
                close_short(name_1, lot_size_1, position.ticket)
                
            if position_2.type == 0:
                close_long(name_2, lot_size_2, position.ticket)
            elif position_2.type == 1:
                close_short(name_1, lot_size_1, position.ticket)    
                    
            send_telegram_message(f'Closed High Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Closed High Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')       
            position_type = FREE
        
    elif position_type == LOW:
        if todays_prices[-2] < tp.iloc[-2] and todays_prices[-1] >= tp.iloc[-1]:
            position_1 = mt5.positions_get()[0]
            position_2 = mt5.positions_get()[1]
            
            if position_1.type == 0:
                close_long(name_2, lot_size_2, position_1.ticket)
            elif position_1.type == 1:
                close_short(name_1, lot_size_1, position_1.ticket)
                
            if position_2.type == 0:
                close_long(name_2, lot_size_2, position_2.ticket)
            elif position_2.type == 1:
                close_short(name_1, lot_size_1, position_2.ticket)
            
            send_telegram_message(f'Closed Low Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Closed Low Positions at: {data.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            position_type = FREE
                     
    return position_type
    

In [97]:
##################################################
# Generate a random artificial asset

end_date = datetime.now()
start_date = end_date - BDay(4)

mt5.initialize()
#location = "Retail\\Forex"
location = "Retail\\ETFs"

all_pairs = get_pairs(location)

# Test for cointegration
candidate_stationary_pairs = first_sieve(all_pairs, start_date, end_date)
print(len(candidate_stationary_pairs))
stationary_pairs = second_sieve(candidate_stationary_pairs, start_date, end_date)
print(len(stationary_pairs))
consistent_pairs = third_seive(stationary_pairs, start_date, end_date)
print(len(consistent_pairs))

##################################################

100%|██████████████████████████████████████████████████████████████████████████████| 5151/5151 [02:21<00:00, 36.35it/s]


68


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:03<00:00, 18.20it/s]


25


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 40.09it/s]

25


In [98]:
for pair in consistent_pairs:
    print(pair['coint_pair'][0].name, pair['coint_pair'][1].name)

Invesco_DB_Ag_Fund_(DBA.P) MSCI_Hong_Kong_ETF_(EWH.P)
Invesco_DB_Ag_Fund_(DBA.P) MSCI_Sth_Africa_ETF_(EZA.P)
Invesco_DB_Ag_Fund_(DBA.P) iShares_China_(MCHI.O)
Invesco_DB_Ag_Fund_(DBA.P) 3D_Printing_ETF_(PRNT.P)
Invesco_DB_Ag_Fund_(DBA.P) iShares_Short_T-Bond_(SHV.O)
Invesco_DB_Ag_Fund_(DBA.P) iShares_Semicond._(SOXX.O)
Invesco_DB_Ag_Fund_(DBA.P) iShares_Thailand_ETF_(THD.P)
EM_Dividend_ETF_(EDIV.P) iShares_Peru_ETF_(EPU.P)
EM_Dividend_ETF_(EDIV.P) MSCI_Sth_Africa_ETF_(EZA.P)
EM_Dividend_ETF_(EDIV.P) Global_Carbon_ETF_(KRBN.P)
EM_Dividend_ETF_(EDIV.P) VanEck_Agribusiness_(MOO.P)
EM_Dividend_ETF_(EDIV.P) MSCI_Norway_ETF_(NORW.P)
EM_Dividend_ETF_(EDIV.P) iShares_Short_T-Bond_(SHV.O)
EM_Dividend_ETF_(EDIV.P) MSCI_India_SC_ETF_(SMIN.N)
EM_Dividend_ETF_(EDIV.P) iShares_Thailand_ETF_(THD.P)
iShares_Taiwan_ETF_(EWT.P) India_50_ETF_(INDY.O)
MSCI_South_Korea_ETF_(EWY.P) MSCI_Sth_Africa_ETF_(EZA.P)
MSCI_Sth_Africa_ETF_(EZA.P) India_50_ETF_(INDY.O)
MSCI_Sth_Africa_ETF_(EZA.P) iShares_Short_T-Bond_

In [92]:
#filtered_pair = []

## Filter out TRY
#for pair in consistent_pairs:
#    if 'TRY' in pair['coint_pair'][0].name or 'TRY' in pair['coint_pair'][1].name:
#        continue
#    else:
#        filtered_pair.append(pair)


# Construct a randomly sampled artificial asset
rand_index = randint(0, len(consistent_pairs) - 1)
pair = consistent_pairs[rand_index]

pair_close = get_pairs_close(pair["coint_pair"], start_date, end_date)
beta = pair["slope"]
artificial_asset = pair_close["close0"] - beta * pair_close["close1"]

ValueError: empty range for randrange() (0, 0, 0)

In [ ]:
for pair in filtered_pair:
    print(pair['coint_pair'][0].name, pair['coint_pair'][1].name)

In [ ]:
symbol_1_csv = pair['coint_pair'][0].name + '.csv'
symbol_2_csv = pair['coint_pair'][1].name + '.csv'
aa_csv = 'artificial_asset.csv'

pair_close["close0"].to_csv(symbol_1_csv, index=False)
pair_close["close1"].to_csv(symbol_2_csv, index=False)
artificial_asset.to_csv(aa_csv, index=False)

data_1 = pd.read_csv(symbol_1_csv)
data_2 = pd.read_csv(symbol_2_csv)
aa_data = pd.read_csv(aa_csv)

In [ ]:
##################################################
# Trade Loop

position_type = 0
while True:
    last_bar_1 = data_1.tail(1)
    last_bar_2 = data_2.tail(1)
    
    new_bar_1, new_bar_2 = new_bar_event_pairs(
        pair['coint_pair'][0].name,
        pair['coint_pair'][1].name, 
        last_bar_1, 
        last_bar_2)
    
    aa_new_bar = new_bar_1 + beta * new_bar_2
    
    new_bar_1.to_csv(symbol_1_csv, mode='a', index=False, header=False)
    new_bar_2.to_csv(symbol_2_csv, mode='a', index=False, header=False)
    aa_new_bar.to_csv(aa_csv, mode='a', index=False, header=False)
        
    aa_data = pd.read_csv(aa_csv)

    lot_size_1 = get_num_of_lots(pair['coint_pair'][0].name, leverage=30, risk=0.005)
    lot_size_2 = get_num_of_lots(pair['coint_pair'][1].name, leverage=30, risk=0.005)
    
    print(f'NEW BAR: {new_bar_1["time"].iloc[0]}')
    
    trade_engine(
        aa_data,
        pair['coint_pair'][0].name,
        pair['coint_pair'][1].name,
        lot_size_1,
        lot_size_2,
        position_type)
    
##################################################